In [1]:
# Set AWS credentials if not using Amazon SageMaker notebook
import os
# os.environ['AWS_ACCESS_KEY_ID'] = '' # Set the access key
# os.environ['AWS_SECRET_ACCESS_KEY'] = '' # Set the secret key
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'ap-southeast-2') # set the AWS region

In [2]:
# import os
# import sys

# # Get the directory of the current script
# if '__file__' in globals():
#     current_dir = os.path.dirname(os.path.abspath(__file__))
# else:
#     current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# # Navigate to the directory containing utils.py (one level up)
# utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# # Add to sys.path
# sys.path.insert(0, utils_dir)

# # Now you can import utils
# import utils

In [3]:
import utils

In [4]:
#### Create a sample AWS Lambda function that you want to convert into MCP tools
lambda_resp = utils.create_gateway_lambda("lambda_function_code.zip")

if lambda_resp is not None:
    if lambda_resp['exit_code'] == 0:
        print("Lambda function created with ARN: ", lambda_resp['lambda_function_arn'])
    else:
        print("Lambda function creation failed with message: ", lambda_resp['lambda_function_arn'])

Reading code from zip file
Creating IAM role for lambda function
Attaching policy to the IAM role
Role 'gateway_lambda_iamrole' created successfully: arn:aws:iam::381491838394:role/gateway_lambda_iamrole
Creating lambda function
Lambda function created with ARN:  arn:aws:lambda:ap-southeast-2:381491838394:function:gateway_lambda


In [7]:
#### Create an IAM role for the Gateway to assume
import utils
agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

attaching role policy agentcore-sample-lambdagateway-role
Agentcore gateway role ARN:  arn:aws:iam::381491838394:role/agentcore-sample-lambdagateway-role


Create Amazon Cognito Pool for Inbound authorization to Gateway

In [5]:
# Creating Cognito User Pool 
import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"Client ID: {client_id}")

# Get discovery URL  
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

Creating or retrieving Cognito resources...
Creating new user pool
Domain created as well
User Pool ID: ap-southeast-2_Eebpi0I9u
creating new resource server
Resource server ensured.
creating new m2m client
Client ID: movsh2jqpvfau674l9fj9m74i
https://cognito-idp.ap-southeast-2.amazonaws.com/ap-southeast-2_Eebpi0I9u/.well-known/openid-configuration


Create the Gateway with Amazon Cognito Authorizer for inbound authorization

In [8]:
# CreateGateway with Cognito authorizer without CMK. Use the Cognito user pool created in the previous step
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # Client MUST match with the ClientId configured in Cognito. Example: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='TestGWforLambda',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # The IAM Role must have permissions to create/list/get/delete Gateway 
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway with AWS Lambda target type'
)
print(create_response)
# Retrieve the GatewayID used for GatewayTarget creation
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

{'ResponseMetadata': {'RequestId': '3a342773-6d50-4ac2-8d36-d79a6dd7d899', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Mon, 01 Dec 2025 22:27:18 GMT', 'content-type': 'application/json', 'content-length': '834', 'connection': 'keep-alive', 'x-amzn-requestid': '3a342773-6d50-4ac2-8d36-d79a6dd7d899', 'x-amzn-remapped-x-amzn-requestid': 'a4b2ec2d-0929-43a1-bb7b-1cc32f095d95', 'x-amzn-remapped-content-length': '834', 'x-amzn-remapped-connection': 'keep-alive', 'x-amz-apigw-id': 'U7hrDG0YSwMEpWA=', 'x-amzn-trace-id': 'Root=1-692e1646-08e32d2751ea27635f12ce78', 'x-amzn-remapped-date': 'Mon, 01 Dec 2025 22:27:18 GMT'}, 'RetryAttempts': 0}, 'gatewayArn': 'arn:aws:bedrock-agentcore:ap-southeast-2:381491838394:gateway/testgwforlambda-bkvjefsgah', 'gatewayId': 'testgwforlambda-bkvjefsgah', 'gatewayUrl': 'https://testgwforlambda-bkvjefsgah.gateway.bedrock-agentcore.ap-southeast-2.amazonaws.com/mcp', 'createdAt': datetime.datetime(2025, 12, 1, 22, 27, 18, 510138, tzinfo=tzutc()), 'updatedAt': d

Create an AWS Lambda target and transform into MCP tools

In [9]:
# Replace the AWS Lambda function ARN below
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": lambda_resp['lambda_function_arn'], # Replace this with your AWS Lambda function ARN
            "toolSchema": {
                "inlinePayload": [
                    {
                        "name": "get_order_tool",
                        "description": "tool to get the order",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    },                    
                    {
                        "name": "update_order_tool",
                        "description": "tool to update the orderId",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    }
                ]
            }
        }
    }
}

credential_config = [ 
    {
        "credentialProviderType" : "GATEWAY_IAM_ROLE"
    }
]
targetname='LambdaUsingSDK'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='Lambda Target using SDK',
    targetConfiguration=lambda_target_config,
    credentialProviderConfigurations=credential_config)

Calling Bedrock AgentCore Gateway from a Strands Agent

In [10]:
import time
time.sleep(10)

In [11]:
print("Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propogation completes")
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("Token response:", token)

Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propogation completes
movsh2jqpvfau674l9fj9m74i
Token response: eyJraWQiOiJFeURUdEtGRDJUQ1IxSEdxc1F1RWhpS1VGTVhtcVVRZDFuRUlZbm5qbGRJPSIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiJtb3ZzaDJqcXB2ZmF1Njc0bDlmajltNzRpIiwidG9rZW5fdXNlIjoiYWNjZXNzIiwic2NvcGUiOiJzYW1wbGUtYWdlbnRjb3JlLWdhdGV3YXktaWRcL2dhdGV3YXk6d3JpdGUgc2FtcGxlLWFnZW50Y29yZS1nYXRld2F5LWlkXC9nYXRld2F5OnJlYWQiLCJhdXRoX3RpbWUiOjE3NjQ2Mjg0ODAsImlzcyI6Imh0dHBzOlwvXC9jb2duaXRvLWlkcC5hcC1zb3V0aGVhc3QtMi5hbWF6b25hd3MuY29tXC9hcC1zb3V0aGVhc3QtMl9FZWJwaTBJOXUiLCJleHAiOjE3NjQ2MzIwODAsImlhdCI6MTc2NDYyODQ4MCwidmVyc2lvbiI6MiwianRpIjoiYWFiZWY5NDgtNTQ1ZS00ZDFlLWExMzMtNmE0Mjc5NzgxNjBmIiwiY2xpZW50X2lkIjoibW92c2gyanFwdmZhdTY3NGw5Zmo5bTc0aSJ9.X8ad1n3yWEnCU5cqtFwwGO_zAHev_dlfTESDYh_gljjT3XYez-uX2cO16G81lbWUPrEaSRLRpygGjioDMnCSD980uTwKC0DMdEk4e-RxWgTjgXqDVN2TWG3AcRG46Ng8EuXBS8rj3RT5X1gxOLGpLeoBY0ofU_fiTJc_7rET4UWJIBXsxlk_SN3aacNnTfj9-9Z6IVjStB9mFzrcME-i0bHkxI

Strands agent calling MCP tools of AWS Lambda using Bedrock AgentCore Gateway

In [12]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

## The IAM credentials configured in ~/.aws/credentials should have access to Bedrock model
yourmodel = BedrockModel(
    model_id="amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [15]:
from strands import Agent
import logging


# Configure the root strands logger. Change it to DEBUG if you are debugging the issue.
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # Call the listTools 
    tools = client.list_tools_sync()
    # Create an Agent with the model and tools
    agent = Agent(model=yourmodel,tools=tools) ## you can replace with any model you like
    print(f"Tools loaded in the agent are {agent.tool_names}")
    # print(f"Tools configuration in the agent are {agent.tool_config}")
    # Invoke the agent with the sample prompt. This will only invoke  MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.
    agent("Hi , can you list all tools available to you")
    # Invoke the agent with sample prompt, invoke the tool and display the response
    agent("Check the order status for order id 123 and show me the exact response from the tool")
         

Tools loaded in the agent are ['LambdaUsingSDK___get_order_tool', 'LambdaUsingSDK___update_order_tool']
<thinking> The user has asked for a list of all available tools. I should provide this information directly as it is part of my initial instructions. </thinking>

Here are the tools available to me:

1. **LambdaUsingSDK___get_order_tool**
   - Description: Tool to get the order.
   - Parameters:
     - `orderId` (string): Property orderId.
   - Required: `orderId`.

2. **LambdaUsingSDK___update_order_tool**
   - Description: Tool to update the orderId.
   - Parameters:
     - `orderId` (string): Property orderId.
   - Required: `orderId`.<thinking> The user has requested to check the order status for a specific order ID. I should use the `LambdaUsingSDK___get_order_tool` to retrieve the order status. </thinking>

Tool #1: LambdaUsingSDK___get_order_tool
<thinking> The tool has returned the order status for order ID 123. I should present this information to the user. </thinking>

The 

In [16]:
 # Call the MCP tool explicitly. The MCP Tool name and arguments must match with your AWS Lambda function or the OpenAPI/Smithy API
with client:    
   result = client.call_tool_sync(
    tool_use_id="get-order-id-123-call-1", # You can replace this with unique identifier. 
    name=targetname+"___get_order_tool", # This is the tool name based on AWS Lambda target types. This will change based on the target name
    arguments={"orderId": "123"}
    )
   # Print the MCP Tool response
   print(f"Tool Call result: {result['content'][0]['text']}")

Tool Call result: {"statusCode":200,"body":"Order Id 123 is in shipped status"}


Clean up  
Additional resources are also created like IAM role, IAM Policies, Credentials provider, AWS Lambda functions, Cognito user pools, s3 buckets that you might need to manually delete as part of the clean up. This depends on the example you run.

In [17]:
import utils
utils.delete_gateway(gateway_client,gatewayID)

Deleting all targets for gateway testgwforlambda-bkvjefsgah
Deleting target  TUDOT8PBQH
Deleting gateway  testgwforlambda-bkvjefsgah
